In [1]:
#-------- Import Libraries --------#
import torch
import os
import sys
import random
import pickle
import mlflow
import numpy as np
import pandas as pd
import torch.nn as nn
import seaborn as sn
import matplotlib.pyplot as plt
from datetime import date
from sklearn.metrics import matthews_corrcoef
import torch.optim as optim  # For all Optimization algorithms, SGD, Adam, etc.
import torch.nn.functional as F  # All functions that don't have any parameters
from sklearn.metrics import accuracy_score, roc_auc_score, roc_curve, auc

In [2]:
#-------- Import Modules from project--------#
import encoding as enc
from model import Net, Net_thesis, Net_project
import functions as func

In [3]:
#-------- Set Device --------#

if torch.cuda.is_available():
    device = torch.device('cuda')
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
else:
    print('No GPUs available. Using CPU instead.')
    device = torch.device('cpu')

No GPUs available. Using CPU instead.


In [3]:
#-------- Seeds --------#

seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

torch.use_deterministic_algorithms(True)

In [4]:
#-------- Directories --------#

DATADIR = '/data/'
TRAINDIR = '../data/train'
VALIDATIONDIR = '../data/validation'
MATRICES = '/data/Matrices'


In [6]:
#-------- Unzip Train --------#

try:
    if len(os.listdir(TRAINDIR)) != 0:
        print("{} already unzipped.".format(TRAINDIR))
except:
    !unzip ../data/train.zip -d ../data/train

    
#-------- Unzip Validation --------#


try:
    if len(os.listdir(VALIDATIONDIR)) != 0:
        print("{} already unzipped.".format(VALIDATIONDIR))
except:
    !unzip ../data/validation.zip -d ../data/validation
    
print('Train directory:\n\n', '\n'.join(str(p) for p in os.listdir(TRAINDIR)), '\n\n')
print('Validation directory:\n\n','\n'.join(str(p) for p in os.listdir(VALIDATIONDIR)))

../data/train already unzipped.
../data/validation already unzipped.
Train directory:

 P1_input.npz
P1_labels.npz
P2_input.npz
P2_labels.npz
P3_input.npz
P3_labels.npz
P4_input.npz
P4_labels.npz
__MACOSX 


Validation directory:

 P5_input.npz
P5_labels.npz
__MACOSX


## Functions

In [14]:
def prepare_data_pca(embedded_list):
    """
    Get result from embedded to have the proper size in 2D 
    ready to run the PCA analysis afterwards.
    """
    n_observations = len(embedded_list)
    n_residues = len(embedded_list[0])
    embedded_att = len(embedded_list[0][0])
    embedded_matrix = torch.tensor(embedded_list).reshape(n_observations * n_residues, embedded_att).numpy()
    return n_observations, embedded_matrix

In [15]:
from sklearn.decomposition import PCA
def run_PCA(data, variance_required = 0.9, max_components = 100):
    """
    Run PCA and get the minimum number of components required to reach 
    the minimum variance required.
    """
    first_model = PCA(n_components = max_components)
    first_model.fit_transform(data)

    variances = first_model.explained_variance_ratio_.cumsum()
    optimal_components = np.argmax(variances > variance_required)

    reduced_model = PCA(n_components = optimal_components)
    fitted_data = reduced_model.fit_transform(data)

    return variances, optimal_components, reduced_model, fitted_data

In [26]:
def back_to_tensor_size(matrix, final_size):
    """
    Reshape matrix back to original 3D shape with the reduced dimensionality
    for the embedded variable. *Option to return a matrix instead of tensor
    by adding .numpy() in the end
    """
    final_tensor = torch.tensor(matrix).reshape(final_size[0], final_size[1], final_size[2])
    return final_tensor

## ESM-1b embedding

In [42]:
# work on esm-1b for PCA
embedding_dir= '../data/embeddedFiles/'; embedding = 'esm-1b'
data_list_embedded = [] 
for i in range(3):
    print("Writing parititon", i + 1)
    infile = open(embedding_dir + 'dataset-{}_{}.pkl'.format(embedding, i), 'rb')
    data_list_embedded.append(pickle.load(infile))
    infile.close()

Writing parititon 1
Writing parititon 2
Writing parititon 3


In [30]:
# doing PCA on our data
final_matrix = np.empty((0, len(data_list_embedded[0][0][0]))); total_observations = 0
for partition in range(3):
    n_obser, partition_matrix = prepare_data_pca(data_list_embedded[partition])
    final_matrix = np.append(final_matrix, partition_matrix, axis = 0)
    total_observations += n_obser
    print(final_matrix.shape)

(642600, 1280)


In [31]:
var_vector, number_components, model, fitted_train = run_PCA(final_matrix)
# and now this model will be applied to validation dataset by running command model.transfrom(X_val)

In [27]:
esm_1b_tensor = back_to_tensor_size(fitted_train, [total_observations, len(data_list_embedded[0][0]), number_components])

## ASM embedding

In [29]:
# now for the ASM embedding
embedding = "esm_ASM"
infile = open(embedding_dir + 'dataset-{}.pkl'.format(embedding), 'rb')
data_ASM_embedded = pickle.load(infile)
infile.close()

In [38]:
# doing PCA on our data

esm_ASM_matrix = np.empty((0, len(data_ASM_embedded[0][0][0]))); total_observations_ASM = 0
for partition in range(3):
    n_obser, partition_matrix = prepare_data_pca(data_ASM_embedded[partition])
    esm_ASM_matrix = np.append(esm_ASM_matrix, partition_matrix, axis = 0)
    print(esm_ASM_matrix.shape)
    total_observations_ASM += n_obser

var_vector_ASM, number_components_ASM, model_ASM, fitted_train_ASM = run_PCA(esm_ASM_matrix)

esm_ASM_tensor = back_to_tensor_size(fitted_train_ASM, [total_observations_ASM, len(data_ASM_embedded[0][0]), number_components_ASM])

(640920, 768)
(1131480, 768)
(1753080, 768)


ValueError: Expected 2D array, got 1D array instead:
array=[-9.02885675e-01 -2.57112563e-01  6.08861864e-01  2.04494047e+00
 -3.94119620e-01  9.94090736e-02  2.52101451e-01  6.98611081e-01
  3.76258284e-01 -5.51021755e-01  8.91607761e+00 -5.86801358e-02
  9.33266699e-01  9.89569664e-01 -6.80327177e-01 -2.65359372e-01
  2.93442547e-01  6.01338243e+00 -6.66363299e-01  9.93139893e-02
 -3.47648501e-01  7.80296326e-02 -1.91798612e-01  9.09580290e-02
 -6.40300870e-01  1.98617592e-01  2.25818455e-02 -3.57028067e-01
  9.09937501e-01  1.53998449e-01  9.24888730e-01 -8.34516764e-01
  6.48360133e-01 -9.47537303e-01 -8.57919455e-01 -5.88815957e-02
 -3.13379735e-01 -1.62997097e-01  1.88168511e-02 -3.53987664e-01
  2.41288334e-01  3.81852649e-02  8.40624571e-02 -3.54742318e-01
  6.85155094e-02 -6.12714648e-01  1.19219303e+00  1.58587798e-01
  1.65693223e-01 -7.23241508e-01 -6.88836396e-01 -7.11757183e-01
 -1.36612788e-01 -5.34582973e-01 -6.69512868e-01  9.60549563e-02
  8.13812986e-02  3.12858343e-0